# HOMEWORK 3

As a first step, we import all the libraries and packages we need for this homework.
We define a function that install the packages needed, so if your laptot doesn't have it, it starts the installation.

In [1]:
def install(package):
    try:
        import subprocess
        import sys
        subprocess.call([sys.executable, "-m", "pip", "install", package])
        print("Library installed")
    except:
        print("Cannot install the library")

In [2]:
try:
    from langdetect import detect
except:
    print("The library LanfDetect is not in the repository, I will install it")
    install('langdetect')
    from langdetect import detect

try:
    from texttable import Texttable
except:
    print("The library TextTable is not in the repository, I will install it")
    install('texttable')
    from texttable import Texttable


In [3]:
import pandas as pd
import unicodecsv as csv # We also import the unicodecsv library to handle any Unicode objects in the CSV report...as csv.
import nltk
import string
import math
import numpy as np

from nltk import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##STEP 1 - DATA

First of all we have to charge our dataset:

We do this step dropping the column 'Unnamed: 0' , every row without description, with a description that has only a point or the '\\' characters.
Then, we decide to remove the newline character and, after this step, we initialize the language column with a 'nd' value; we will use this column to know the language of the record of our dataset.

In [6]:
DIR = 'C:/Users/david/Desktop/DATA SCIENCE/AMD/documents/' # data/docs/

df = pd.read_csv("Airbnb_Texas_Rentals.csv", sep=",") # we read the dataset

In [7]:
df = df.drop('Unnamed: 0',axis = 1) #this column is useless for our analisys
df = df[pd.notnull((df['description'])) & (df['description'] != "\\" ) & (df['description'] != "." )] #If the row has no description, for us is useless, so we drop it
df = df.replace(r'\\n',' ', regex=True) #Remove all the new lines in the dataframe
df['language'] = 'nd'
df.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,language
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,nd
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,nd
2,$59,1,Houston,January 2017,'River house on island close to the city' A w...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,nd
3,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,nd
4,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,nd


STEP 2 - WRITE DOCUMENTS

Now we use the dataframe to write a file .tsv for each record of our dataset.

In [8]:
for row in df.iterrows():
    file_name = DIR + 'doc_'+str(row[0])+'.tsv'
    with open(file_name,'wb') as file: # The wb means we are writing a file in binary mode.
        csv_writer = csv.writer(file, delimiter = '\t') # Now we are writing the files as tab separated files.
        csv_writer.writerow(list(row[1].values)) # Writerow want a list object ad input, so we used list.
        file.close() # Once the file is written, we close it, so we free some memory


In the next cell we will preprocess our data, and our goal is having clean data.
To clean data, we mean:

Removing stopwords

Removing punctuation

Stemming

We will do this for each language we consider in (english, spanish, german and dutch).
After this step, we should have a dictionary with our words and we are ready to start making search engines.

In [9]:
docdict = {}

languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}

for i, row in df[:].iterrows(): #We limit to 5 just to work quickly

    #Remove punctuation
    text = str(row['description']).translate({key: None for key in string.punctuation}) + ' '+ str(row['title']).translate({key: None for key in string.punctuation})
    
    if(text):
        try:
            lan = detect(str(text))
            if (lan in languages):
                languages[lan]
        
                #The text here is Spanish, we can decide what to do
                df.loc[i,'language'] = lan
        
                #Remove all the stopwords and all the non alphabetic words
                tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
        
                #Stem all the words
                docdict[i] = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
            else:
                pass
        except:
            pass

# languages
docdict

{0: ['welcom',
  'stay',
  'privat',
  'room',
  'queen',
  'bed',
  'detach',
  'privat',
  'bathroom',
  'second',
  'anoth',
  'privat',
  'bedroom',
  'sofa',
  'bed',
  'avail',
  'addit',
  'addit',
  'iah',
  'airport',
  'airport',
  'avail',
  'privat',
  'iah',
  'airport'],
 1: ['fulli',
  'remodel',
  'home',
  'upscal',
  'nw',
  'alamo',
  'height',
  'amaz',
  'locat',
  'hous',
  'conveni',
  'locat',
  'quiet',
  'beauti',
  'season',
  'prestigi',
  'neighborhood',
  'close',
  'loop',
  'featur',
  'open',
  'floor',
  'origin',
  'hardwood',
  'full',
  'bathroom',
  'independ',
  'room',
  'sleep',
  'european',
  'inspir',
  'kitchen',
  'driveway',
  'park',
  'uniqu',
  'alamo',
  'height',
  'design',
  'inspir'],
 2: ['hous',
  'island',
  'close',
  'well',
  'maintain',
  'river',
  'hous',
  'san',
  'jacinto',
  'river',
  'extra',
  'room',
  'temporari',
  'river',
  'hous',
  'near',
  'citi'],
 3: ['privat',
  'bedroom',
  'cute',
  'littl',
  'home',


Now we want a vocabulary, with a word (in the dictionary we just created) and a id to identificate it.

In [10]:
voc = {}
cont = 0
for key, value in docdict.items():
    for elem in value:
        if(elem not in voc):
            voc[elem] = cont
            cont += 1
print(voc)

{'welcom': 0, 'stay': 1, 'privat': 2, 'room': 3, 'queen': 4, 'bed': 5, 'detach': 6, 'bathroom': 7, 'second': 8, 'anoth': 9, 'bedroom': 10, 'sofa': 11, 'avail': 12, 'addit': 13, 'iah': 14, 'airport': 15, 'fulli': 16, 'remodel': 17, 'home': 18, 'upscal': 19, 'nw': 20, 'alamo': 21, 'height': 22, 'amaz': 23, 'locat': 24, 'hous': 25, 'conveni': 26, 'quiet': 27, 'beauti': 28, 'season': 29, 'prestigi': 30, 'neighborhood': 31, 'close': 32, 'loop': 33, 'featur': 34, 'open': 35, 'floor': 36, 'origin': 37, 'hardwood': 38, 'full': 39, 'independ': 40, 'sleep': 41, 'european': 42, 'inspir': 43, 'kitchen': 44, 'driveway': 45, 'park': 46, 'uniqu': 47, 'design': 48, 'island': 49, 'well': 50, 'maintain': 51, 'river': 52, 'san': 53, 'jacinto': 54, 'extra': 55, 'temporari': 56, 'near': 57, 'citi': 58, 'cute': 59, 'littl': 60, 'situat': 61, 'covet': 62, 'garden': 63, 'acr': 64, 'access': 65, 'campus': 66, 'recent': 67, 'purchas': 68, 'mile': 69, 'outsid': 70, 'downtown': 71, 'fort': 72, 'worth': 73, 'happi

The next step to have a search engine, and before to create it as a function, is to make the inverted index dictionary.
In other words, we want a dictionary that with the 'term_id' as keys and the documents where each word (identified by the term_id) appear as values.

In [11]:
invidx = {} # 1: [0,2,5], 245: [1, 4, 7] ->  1: [(0, 1), (2, 3), 5], 245: [1, 4, 7]

for key, value in docdict.items():
    #
    for elem in value:
        if(voc[elem] in invidx):
            invidx[voc[elem]] = invidx[voc[elem]] + [key]
        else:
            invidx[voc[elem]] = [key]
print(invidx)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:

def search_eng(string, n):
    languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}
    text = string
    lan = detect(str(text))
    if(lan not in languages):
        lan = 'en'
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            if(len(listofdocs) == 0):
                listofdocs = set(invidx[voc[elem]])
            else:
                listofdocs = listofdocs & set(invidx[voc[elem]])
    t = Texttable()
    t.add_row(['Title', 'Description', 'City', 'URL'])
    for elem in listofdocs:
        file_name = DIR + 'doc_'+str(elem)+'.tsv'
        with open(file_name,'rb') as file:
            text = str(file.read()).split('\\t')
            t.add_row([text[7], text[4], text[2], text[8]])
            file.close()

    print(t.draw())

Now we are ready to ask our search engine our requests; in this first versions our results are printed without a ranking.

In [16]:
searchJac('Increíble habitación lugares', 10)

+---------------------+---------------------+-------------+--------------------+
| Title               | Description         | City        | URL                |
+---------------------+---------------------+-------------+--------------------+
| Cuarto luminoso     | Lugares de          | Irving      | https://www.airbnb |
|                     | inter\xc3\xa9s:     |             | .com/rooms/1607339 |
|                     | transporte          |             | 3?location=Coppell |
|                     | p\xc3\xbablico,     |             | %2C%20TX           |
|                     | parques. Te va a    |             |                    |
|                     | encantar mi espacio |             |                    |
|                     | porque Hay mucho    |             |                    |
|                     | verde y espacios    |             |                    |
|                     | con juegos para     |             |                    |
|                     | ni\x

Conjunctive query and ranking score

First of all we create two functions; the first one will compute the tfidf of the word in a document. The second one is a function that return the cosine similarity, so defined:

mettere definizione di cosine similarity.

In [17]:
#function tfidf
def tf_idf(word,doc,D,N):
    
    f = doc.count(word) / len(doc)
    tfidf = math.log(D/N)*f
    
    return (tfidf)

#function cosine similarity

def cosine_sim(x , y):
    
    a = dot(x,y)
    
    b = np.linalg.norm(x)
    c = np.linalg.norm(y)
    
    cos = (a/(b*c))
    
    return (cos)
    

    

In [18]:
# Here I have to create the dictionary with the term ID as key and then the documents where the term id is and also the tfidf.

invidx2 = invidx.copy()
alldocs = []
for key, value in invidx2.items():
    alldocs = alldocs + invidx2[key]
    invidx2[key] = set(value)

Ndocs = len(set(alldocs))
print(Ndocs)

18156


In [19]:
inv_voc = {v: k for k, v in voc.items()}
invidx3 = invidx2.copy()

for key, value in invidx2.items():
    tmp = []
    for elem in value:
        file_name = DIR + 'doc_'+str(elem)+'.tsv'
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            text = text[4].split(' ')
            file.close()
        x = tf_idf(inv_voc[key], text, Ndocs, len(value))
        tmp.append((elem,x))
    invidx3[key] = tmp

print(invidx3)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
print(invidx3[1])

[(0, 0.054001120023417906), (8192, 0.02475051334406654), (8204, 0.1131452038585899), (13, 0.027953520953298682), (16401, 0.06988380238324671), (16403, 0.0), (8215, 0.0), (8218, 0.0), (30, 0.028286300964647473), (16417, 0.028976210744273024), (39, 0.02475051334406654), (46, 0.0), (8240, 0.05657260192929495), (8245, 0.03210877406797822), (8246, 0.030462170269620358), (57, 0.028286300964647473), (8266, 0.028286300964647473), (16462, 0.030462170269620358), (16464, 0.05165324523979104), (16466, 0.08485890289394242), (85, 0.02862709977145046), (8277, 0.06600136891751077), (16475, 0.06600136891751077), (8284, 0.0), (100, 0.06600136891751077), (8294, 0.015735425702188), (103, 0.03600074668227861), (104, 0.05280109513400862), (8296, 0.026697182932925705), (109, 0.0), (8301, 0.0), (8303, 0.03085778287052452), (16494, 0.060924340539240715), (8315, 0.033465482831413915), (8319, 0.0), (8325, 0.03210877406797822), (8326, 0.05165324523979104), (8335, 0.0), (144, 0.0), (8336, 0.009542366590483485), (1

In [21]:
import pickle

# with open(DIR+'dicIndex.json', 'w') as file:
#      file.write(json.dumps(invidx3)) # use `json.loads` to do the reverse
        
with open(DIR+'dicIndex.pkl', 'wb') as file:
        pickle.dump(invidx3, file, pickle.HIGHEST_PROTOCOL)

In [22]:
def searchCos(string, n):
    with open(DIR+'dicIndex.pkl', 'rb') as file:
        data = pickle.load(file)
    languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}
    text = string
    lan = detect(str(text))
    if(lan not in languages):
        lan = 'en'
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            if(len(listofdocs) == 0):
                listofdocs = set(invidx3[voc[elem]])
            else:
                listofdocs = listofdocs & set(invidx3[voc[elem]])
    listofdocs = sorted(list(listofdocs), key=lambda x: -x[1])
    t = Texttable()
    t.add_row(['Title', 'Description', 'City', 'URL'])
    for elem in listofdocs:
        file_name = DIR + 'doc_'+str(elem[0])+'.tsv'
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            t.add_row([text[7], text[4], text[2], text[8]])
            file.close()

    print(t.draw())

Now we are ready to do requests to our second search engine, with the results ranked, as we said, by the cosine similarity with our query.

In [23]:
searchCos('Increíble habitación lugares', 10)

+---------------------+---------------------+-------------+--------------------+
| Title               | Description         | City        | URL                |
+---------------------+---------------------+-------------+--------------------+
| Excellent option in | Lugares de          | San Antonio | https://www.airbnb |
| San Antonio, great  | inter\xc3\xa9s: Six |             | .com/rooms/1452903 |
| location            | Flags, Sea World,   |             | 4?location=Convers |
|                     | San Marcos Outlet,  |             | e%2C%20TX          |
|                     | \xc3\x81lamo, San   |             |                    |
|                     | Antonio downtown. , |             |                    |
|                     | restaurantes y      |             |                    |
|                     | comida. Te va a     |             |                    |
|                     | encantar mi lugar   |             |                    |
|                     | debi

STEP 4 - DEFINE A NEW SCORE

STEP BONUS - MAP

In [46]:
import folium
from geopy import distance
import re

In [47]:
m = folium.Map(location = [32.78815, -96.7954], zoom_start = 6)
m

In [31]:
tooltip = 'Click me!'

folium.Marker([32.78810, -96.7984], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(m)
folium.Marker([32.77815, -96.9954], popup='<b>Timberline Lodge</b>', tooltip=tooltip).add_to(m)

m

In [35]:
folium.Circle(
    radius = 500,
    location = [32.78815, -96.7954],
    popup = 'The Waterfront',
    color = 'crimson',
    fill = True,
).add_to(m)

m

In [50]:
def google_maps(c,r):

    for i in range(len(df)):
        doc = csv.reader(DIR + 'doc_'+str(i)+'.tsv', delimiter = '\t')
        doc = str(doc)
        text = re.split(u'\t', doc)
        
        punto_da_confrontare = (text[5],text[6])
        if (distance.distance(c, punto_da_confrontare).meters) < r:
    
            tooltip = 'Click me!'
            folium.Marker([punto_da_confrontare], popup = '<i>Mt. Hood Meadows</i>', tooltip = tooltip).add_to(m)
            
        else:
            pass

In [51]:
google_maps((32.78815, -96.7954), 500)

IndexError: list index out of range

In [53]:
doc = csv.reader(DIR + 'doc_3.tsv', delimiter = '\t')
doc = str(doc)
text = re.split(u'\t', doc)